In [1]:
from flask import Flask,request,render_template
import pickle as pkl
import pandas as pd
import numpy as np

In [2]:
# Create a Web Server Gateway Interface Application.

app = Flask(__name__)

In [3]:
# To display the home page.

@app.route('/', endpoint = 'home', methods = ['GET'])
def home_page():
    return render_template('home.html')

In [4]:
# To load the pickle files of OHE, StandardScaler, Polynomial Regressor, Lasso Regressor.

one_hot_encoder = pkl.load(open('OneHotEncoder.pkl','rb'))
standard_scaler = pkl.load(open('StandardScaler.pkl','rb'))
polynomial_reg  = pkl.load(open('Polynomial.pkl','rb'))
lasso_reg       = pkl.load(open('LassoRegPoly.pkl','rb'))

In [5]:
# To predict the insurance premium cost using Lasso Regression.

@app.route('/predictdata', methods = ['GET','POST'])
def predict_data():
    if request.method == 'POST':
        age = abs(int(request.form.get('age')))
        sex = request.form.get('sex').lower()
        bmi = abs(float(request.form.get('bmi')))
        children = abs(int(request.form.get('children')))
        smoker = request.form.get('smoker').lower()
        region = request.form.get('region').lower()
        
        # Convert the above values into dataframe (for one hot encoding)
        data = pd.DataFrame({
            'age' : [age],
            'sex' : [sex],
            'bmi' : [bmi],
            'children' : [children],
            'smoker' : [smoker],
            'region' : [region]
        })
        
        # Perform one hot encoding.
        cat_cols = ['sex','smoker','region']
        data_encoded = one_hot_encoder.transform(data[cat_cols]).toarray()
        
        # Convert the encoded array into a dataframe (for concat with original dataframe)
        data_encoded_df = pd.DataFrame(data_encoded, columns = one_hot_encoder.get_feature_names_out(cat_cols), index = data.index)
        
        # Concat encoded df with original df.
        data = pd.concat([data,data_encoded_df],axis = 1)
        
        # Drop the categorical features (sex, smoker, region)
        data = data.drop(cat_cols,axis = 1)
        
        # Perform Standardization.
        data_scaled = standard_scaler.transform(data)
        
        # Perform Polynomial Regression.
        data_scaled_poly = polynomial_reg.transform(data_scaled)
        
        # Perform Prediction through Lasso Regression.
        results = lasso_reg.predict(data_scaled_poly)
        
        return render_template('Parameters.html', result = round(results[0],2))
    else:
        return render_template('Parameters.html')        

In [6]:
# Run the application.

if __name__ == '__main__':
    app.run(host = '0.0.0.0', port = 8080)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.29.125:8080
Press CTRL+C to quit
127.0.0.1 - - [05/Feb/2025 13:27:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2025 13:27:41] "GET /predictdata HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2025 13:28:05] "POST /predictdata HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2025 13:28:37] "POST /predictdata HTTP/1.1" 200 -
C:\Users\003HL1744\AppData\Local\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
127.0.0.1 - - [05/Feb/2025 19:17:37] "POST /predictdata HTTP/1.1" 200 -
C:\Users\003HL1744\AppData\Local\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
127.0.0.1 - - [05/Feb/2025 19:1

In [ ]:
# Points to Note:
# 1. While creating dataframe, the values of the columns should be passed in a list. Otherwise, the error 
# ValueError: If using all scalar values, you must pass an index will be returned.
# 2. Feature names should be same as when the model is trained.